## **Loading Libraries**

In [ ]:
#load the basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')
import os
import tensorflow as tf
import keras
import cv2

#load libraries for data preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

#load libraries for building CNN
from tensorflow.keras.layers import *
from tensorflow.keras import layers , models, optimizers
from tensorflow.keras.models import Sequential, Model

#load libraries for ResNet transfer learning
from tensorflow.keras.applications import ResNet50V2

#load libraries for callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

#load libraries for visualizing
from tensorflow.keras.utils import plot_model



## Analysizing Dataset 

In [ ]:
# define a  function to caculate the number of each emotion classes
train_dir = 'FER2013/train/'
test_dir = 'FER2013/test/'

def Classes_Count( path, name):
    Classes_Dict = {}

    for Class in os.listdir(path):

        Full_Path = os.path.join(path, Class)
        Classes_Dict[Class] = len(os.listdir(Full_Path))

    df = pd.DataFrame(Classes_Dict, index=[name])

    return df

Train_Count = Classes_Count(train_dir, 'Train').transpose().sort_values(by="Train", ascending=False)
Test_Count = Classes_Count(test_dir, 'Test').transpose().sort_values(by="Test", ascending=False)

# Data Visualization

### **Emotion Distribution Table**

In [ ]:
pd.concat([Train_Count,Test_Count] , axis=1)

# Data Visualization

### **Bar chart of the train and test data**

In [ ]:
Train_Count.plot(kind='barh')
plt.title('Train Data Distribution')
plt.xlabel('Number of images')
plt.ylabel('Emotions')
plt.show()

In [ ]:
Test_Count.plot(kind='barh')

### **Show a batch of images**

In [ ]:
plt.style.use('default')
plt.figure(figsize = (25, 8))
image_count = 1
BASE_URL = 'FER2013/train/'

for directory in os.listdir(BASE_URL):
    if directory[0] != '.':
        for i, file in enumerate(os.listdir(BASE_URL +'/'+ directory)):
            if i == 1:
                break
            else:
                fig = plt.subplot(1, 7, image_count)
                image_count += 1
                image = cv2.imread(BASE_URL + directory + '/' + file)
                plt.imshow(image)
                plt.title(directory, fontsize = 20)

# Data Preprocessing

###   **Handling Missing Data**

In [ ]:
merged_df = pd.concat([Train_Count, Test_Count], axis=1)
null_values = merged_df.isnull().sum()
print(null_values)

There are no null values in this dataset.

###   **Data Augmentation**

In [ ]:
img_shape = 48
batch_size = 64
train_data_path = 'FER2013/train/'
test_data_path = 'FER2013/test/'

In [ ]:
#using data generators to generate batches of images
train_preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
        # Data Augmentation
        rotation_range=10,
        zoom_range=0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest',
    )

test_preprocessor = ImageDataGenerator(
    rescale = 1 / 255.,
)

train_data = train_preprocessor.flow_from_directory(
    train_data_path,
    class_mode="categorical",
    target_size=(img_shape,img_shape),
    color_mode='rgb',
    shuffle=True,
    batch_size=batch_size,
    subset='training',
)

test_data = test_preprocessor.flow_from_directory(
    test_data_path,
    class_mode="categorical",
    target_size=(img_shape,img_shape),
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size,
)

# ResNet Model Creation

In [ ]:
# specifing new image shape for resnet
img_shape = 224
batch_size = 64
train_data_path = 'FER2013/train/'  # 80%
test_data_path = 'FER2013/test/'    # 20%

In [ ]:
#load data again
train_preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
        rotation_range=10,
        zoom_range=0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest',
    )


test_preprocessor = ImageDataGenerator(
    rescale = 1 / 255.,
)

train_data = train_preprocessor.flow_from_directory(
    train_data_path,
    class_mode="categorical",
    target_size=(img_shape,img_shape),
    color_mode='rgb',
    shuffle=True,
    batch_size=batch_size,
    subset='training',
)

test_data = test_preprocessor.flow_from_directory(
    test_data_path,
    class_mode="categorical",
    target_size=(img_shape,img_shape),
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size,
)

### **ResNet50V2 Definition**

In [ ]:
# 224,224,3
ResNet50V2 = tf.keras.applications.ResNet50V2(input_shape=(224, 224, 3),
                                               include_top= False,
                                               weights='imagenet'
                                               )

#ResNet50V2.summary()

In [ ]:
# Freezing all layers except last 50

ResNet50V2.trainable = True

for layer in ResNet50V2.layers[:-50]:
    layer.trainable = False

In [ ]:
def Create_ResNet50V2_Model():

    model = Sequential([
                      ResNet50V2,
                      Dropout(.25),
                      BatchNormalization(),
                      Flatten(),
                      Dense(64, activation='relu'),
                      BatchNormalization(),
                      Dropout(.5),
                      Dense(7,activation='softmax')
                    ])
    return model

In [ ]:
ResNet50V2_Model = Create_ResNet50V2_Model()

ResNet50V2_Model.summary()

ResNet50V2_Model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### **Specifying Callbacks and Early Stopping**

In [ ]:
# Create Callback Checkpoint
checkpoint_path = "ResNet50V2_Model_Checkpoint"
Checkpoint = ModelCheckpoint(checkpoint_path, monitor="val_accuracy", save_best_only=True)

# Create Early Stopping Callback to monitor the accuracy
Early_Stopping = EarlyStopping(monitor = 'val_accuracy', patience = 7, restore_best_weights = True, verbose=1)

# Create ReduceLROnPlateau Callback to reduce overfitting by decreasing learning
Reducing_LR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                  factor=0.2,
                                                  patience=2,
#                                                   min_lr=0.00005,
                                                  verbose=1)

callbacks = [Early_Stopping, Reducing_LR]
steps_per_epoch = train_data.n // train_data.batch_size
validation_steps = test_data.n // test_data.batch_size

### **ResNet Training**

In [ ]:
ResNet50V2_history = ResNet50V2_Model.fit(train_data ,validation_data = test_data , epochs=30, batch_size=batch_size,
                                         callbacks = callbacks, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)

### **Visualising ResNet Loss and Accuracy**

In [ ]:
def plot_curves(history):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]
    epochs = range(len(history.history["loss"]))
    plt.figure(figsize=(15,5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label = "training_loss")
    plt.plot(epochs, val_loss, label = "val_loss")
    plt.title("Loss")
    plt.xlabel("epochs")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label = "training_accuracy")
    plt.plot(epochs, val_accuracy, label = "val_accuracy")
    plt.title("Accuracy")
    plt.xlabel("epochs")
    plt.legend()

In [ ]:
plot_curves(ResNet50V2_history)

In [ ]:
ResNet50V2_Score = ResNet50V2_Model.evaluate(test_data)

print("    Test Loss: {:.5f}".format(ResNet50V2_Score[0]))
print("Test Accuracy: {:.2f}%".format(ResNet50V2_Score[1] * 100))

### **ResNet Confusion Matrix**

In [ ]:
ResNet50V2_Predictions = ResNet50V2_Model.predict(test_data)

# Choosing highest probalbilty class in every prediction
ResNet50V2_Predictions = np.argmax(ResNet50V2_Predictions, axis=1)

In [ ]:
import seaborn as sns 
from sklearn.metrics import confusion_matrix

fig , ax= plt.subplots(figsize=(15,10))

cm=confusion_matrix(test_data.labels, ResNet50V2_Predictions)

sns.heatmap(cm, annot=True, fmt='g', ax=ax)

ax.set_xlabel('Predicted labels',fontsize=15, fontweight='bold')
ax.set_ylabel('True labels', fontsize=15, fontweight='bold')
ax.set_title('ResNet50V2 Confusion Matrix', fontsize=20, fontweight='bold')

# Evaluation

### **Shuffling Test Data**

In [ ]:
Emotion_Classes = ['Angry',
                  'Disgust',
                  'Fear',
                  'Happy',
                  'Neutral',
                  'Sad',
                  'Surprise']

In [ ]:
# Shuffling Test Data to show diffrent classes
test_preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
    )

test_generator = test_preprocessor.flow_from_directory(
    test_data_path,
    class_mode="categorical",
    target_size=(48,48),
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size,
)

### **ResNet50V2 Predictions**

In [ ]:
# Shuffling Test Data to show diffrent classes
test_preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
    )

test_generator = test_preprocessor.flow_from_directory(
    test_data_path,
    class_mode="categorical",
    target_size=(224,224),
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size,
)

In [ ]:
# Display 10 random pictures from the dataset with their labels
Random_batch = np.random.randint(0, len(test_generator) - 1)
Random_Img_Index = np.random.randint(0, batch_size - 1 , 10)

fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(25, 10),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):

    Random_Img = test_generator[Random_batch][0][Random_Img_Index[i]]
    Random_Img_Label = np.argmax(test_generator[Random_batch][1][Random_Img_Index[i]])
    Model_Prediction = np.argmax(ResNet50V2_Model.predict( tf.expand_dims(Random_Img, axis=0) , verbose=0))
    ax.imshow(Random_Img)

    if Emotion_Classes[Random_Img_Label] == Emotion_Classes[Model_Prediction]:
          color = "green"
    else:
          color = "red"
    ax.set_title(f"True: {Emotion_Classes[Random_Img_Label]}\nPredicted: {Emotion_Classes[Model_Prediction]}", color=color)
    
plt.show()
plt.tight_layout()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


y_true = test_data.labels

# Calculate accuracy
accuracy = accuracy_score(y_true, ResNet50V2_Predictions)

# Calculate precision
precision = precision_score(y_true, ResNet50V2_Predictions, average='macro')

# Calculate recall
recall = recall_score(y_true, ResNet50V2_Predictions, average='macro')

# Calculate F1 score
f1 = f1_score(y_true, ResNet50V2_Predictions, average='macro')


# Print the metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Error rate:",1-accuracy)

## **Save Model**

In [ ]:
ResNet50V2_Model.save("ResNet50V2_Model.h5")

In [1]:
import cv2
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt
from IPython.display import display, Image
import ipywidgets as widgets
import threading

# Load the ResNet50V2 model
def load_resnet_model(model_path):
    return load_model(model_path)

# Load the Haar Cascade model for face detection
def load_face_cascade():
    return cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the webcam
def initialize_webcam(width=640, height=480):
    cap = cv2.VideoCapture(0)
    cap.set(3, width)
    cap.set(4, height)
    return cap

# Detect faces in the frame
def detect_faces(frame, face_cascade):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Preprocess face image
def preprocess_face(face):
    face = cv2.resize(face, (224, 224))
    face = face.astype('float32') / 255.0
    return np.expand_dims(face, axis=0)

# Predict emotion using ResNet50V2 model
def predict_emotion(model, face):
    return np.argmax(model.predict(face))

# Draw rectangles around detected faces and display predicted emotions
def draw_faces(frame, faces, emotions):
    for (x, y, w, h), emotion in zip(faces, emotions):
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    return frame

# Main function to run emotion detection
def run_emotion_detection(ResNet_model, face_cascade):
    cap = initialize_webcam()
    emotion_counts = {emotion: 0 for emotion in Emotion_Classes}

    while True:
        ret, frame = cap.read()
        if cv2.waitKey(1) & 0xFF == ord('q'):  # quit when 'q' is pressed
            cap.release()
            break

        faces = detect_faces(frame, face_cascade)

        emotions = []
        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            face = preprocess_face(face)
            emotion = predict_emotion(ResNet_model, face)
            emotions.append(Emotion_Classes[emotion])
            emotion_counts[Emotion_Classes[emotion]] += 1

        frame = draw_faces(frame, faces, emotions)
        cv2.imshow('Emotion Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    for emotion, count in emotion_counts.items():
        print(f'{emotion}: {count} frames')

    overall_emotion = max(emotion_counts, key=emotion_counts.get)
    print(f'Overall Emotion: {overall_emotion}')

# Example usage
if __name__ == "__main__":
    Emotion_Classes = ['Angry',
                  'Disgust',
                  'Fear',
                  'Happy',
                  'Neutral',
                  'Sad',
                  'Surprise']
    ResNet50V2_Model = load_resnet_model('ResNet50V2_Model.h5')
    face_cascade = load_face_cascade()
    run_emotion_detection(ResNet50V2_Model, face_cascade)


1/1 [==============================] - 0s 100ms/step


1/1 [==============================] - 0s 126ms/step
Angry: 7 frames
Disgust: 0 frames
Fear: 0 frames
Happy: 7 frames
Neutral: 141 frames
Sad: 0 frames
Surprise: 20 frames
Overall Emotion: Neutral


In [ ]:
pip uninstall opencv-python 

In [ ]:
pip install opencv-python

In [ ]:
pip uninstall opencv-python-headless -y 

In [ ]:
pip install opencv-python --upgrade